In [1]:
#-*- coding: utf-8 -*-
import requests
from dotenv import load_dotenv
import os, sys, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px

from glob import glob
from tqdm.auto import tqdm

from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import ColumnDataSource, HoverTool, BoxAnnotation
from bokeh.palettes import Category10
from bokeh.layouts import layout
from bokeh.io.export import export_png

sys.path.append(os.path.abspath("../../code"))

import functions as fc
import naverApi_function as nv

# pandas 설정
pd.set_option('display.max_columns', 100)

# matplotlib 설정
plt.rcParams['font.family'] = 'NanumGothic'  # 나눔고딕 설정
plt.rcParams['axes.unicode_minus'] = False   # 마이너스 기호 깨짐 방지

load_dotenv()

client_id = os.getenv("naver_client_id")
client_secret = os.getenv("naver_client_secret")

In [2]:
# API 설정
api_config = nv.NaverAPIConfig(
    client_id=client_id,
    client_secret=client_secret
)

# API 클라이언트 생성
client = nv.NaverShoppingInsight(api_config)

In [3]:
# 2. 모듈 import
from naverApi_function import (
    TimeUnit,
    NaverAPIConfig,
    NaverShoppingInsight,
    CategoryRequestConfig, DeviceRequestConfig, GenderRequestConfig, AgeRequestConfig, KeywordRequestConfig, generate_monthly_dates, plot_keyword_age_trends
)

# 4. API 클라이언트 생성
client = NaverShoppingInsight(api_config)

# 1. 쇼핑인사이트

## 1.1 분야별 트렌드 조회

In [ ]:
# 5. 요청 설정 (쇼핑인사이트 분야별 트렌드 조회)
category_config = CategoryRequestConfig(
    start_date="2025-01-01",
    end_date="2025-03-15",
    categories=[
        {"name": "농산물", "param": ["50000160"]},
        {"name": "수산물", "param": ["50000159"]}
    ],
    ages=["20", "30", "40"],
    category="50000006"
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends = client.fetch_shopping_trends(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")

## 1.2 분야 내 기기별 트렌드 조회

In [15]:
# 분야 내 기기별 트렌드 조회
category_config = DeviceRequestConfig(
    start_date="2025-01-01",
    end_date="2025-03-15",
    category="50000006",
    gender="f",
    ages=["20", "30", "40"],
    time_unit=TimeUnit.MONTH
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends = client.fetch_device_traffic(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")

카테고리 분석 결과:
      period       pc     mobile
0 2025-01-01  8.12360  100.00000
1 2025-02-01  7.56772   84.72968
2 2025-03-01  7.35977   84.46491


## 1.3 분야 내 성별 트렌드 조회

In [27]:
# 분야 내 성별 트렌드 조회
category_config = GenderRequestConfig(
    start_date="2025-01-01",
    end_date="2025-03-15",
    category="50000006",
    ages=["20", "30", "40"],
    time_unit=TimeUnit.MONTH
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends = client.fetch_gender_traffic(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")

API 호출 성공.


## 1.4 분야 내 연령별 트렌드 조회

In [4]:
# 분야 내 연령별 트렌드 조회
category_config = AgeRequestConfig(
    start_date="2025-01-01",
    end_date="2025-03-15",
    category="50000006",
    ages=["20", "30", "40"],
    time_unit=TimeUnit.MONTH
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends = client.fetch_age_traffic(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")

카테고리 분석 결과:
      period 10대       20대       30대        40대 50대 60대
0 2025-01-01      23.58498  72.22454  100.00000        
1 2025-02-01      21.55020  61.69074   83.20083        
2 2025-03-01      22.09461  63.38663   85.17130        


# 2. 키워드

## 2.1 키워드 별 트렌드 조회

In [6]:
keywords = [
    {"name": "식품/축산물", "param": [ "축산물"]},
    {"name": "식품/수산물", "param": [ "수산물"]},
    {"name": "식품/농산물", "param": [ "농산물"]}
]

category_config = KeywordRequestConfig(
    start_date="2022-01-01",
    end_date="2025-03-15",
    category="50000006",
    keyword=keywords,
    time_unit=TimeUnit.DATE
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_keyword_trends = client.fetch_keyword_trends(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")

API 호출 성공.


In [7]:
df_keyword_trends

,period,식품/축산물,식품/수산물,식품/농산물
0,2022-01-04,0.55776,2.39043,0.55776
1,2022-01-05,0.15936,2.31075,1.75298
2,2022-01-08,0.07968,0.71713,1.59362
3,2022-01-10,0.23904,3.26693,0.55776
4,2022-01-11,0.15936,2.39043,0.87649
...,...,...,...,...
287,2025-02-18,0.07968,1.91235,1.11553
288,2025-02-25,0.55776,2.54980,0.39840
289,2025-02-28,0.07968,1.19521,0.63745
290,2025-03-14,0.07968,1.43426,0.23904


In [15]:
output_notebook()

p = figure(
    title="네이버 쇼핑 키워드 별 트렌드 조회", 
    x_axis_type="datetime",
    height=600,
    width=1200,
    tools="pan,box_zoom,reset,save",
    sizing_mode="stretch_width"
)

palette = Category10[3]

for i, column in enumerate(sorted(df_keyword_trends.columns[1:])):
    source = ColumnDataSource(df_keyword_trends[['period', column]])
    
    line = p.line('period', column, 
                  line_width=2, 
                  color=palette[i],
                  legend_label=column,
                  source=source)
                  
    p.scatter('period', column,
              size=5, 
              color=palette[i],
              legend_label=column,
              source=source)

    hover = HoverTool(tooltips=[
        ('키워드', column),
        ('비율', f'@{column}{{0.00}}%'),
        ('일자', '@period{%F}')
    ], formatters={'@period': 'datetime'}, renderers=[line])
    
    p.add_tools(hover)

p.xaxis.major_label_orientation = 0.8
p.legend.click_policy = "hide"
p.legend.location = "top_left" 
p.xaxis.axis_label = '날짜'
p.yaxis.axis_label = '검색 비율(%)'

show(p)

Loading BokehJS ...

## 2.2 키워드 내 기기별 트렌드 조회

In [14]:
# ✅ category는 object list 형식이어야 함
keywords = "건강"

category_config = KeywordRequestConfig(
    start_date="2025-01-01",
    end_date="2025-03-15",
    category="50000006",
    keyword=keywords,
    time_unit=TimeUnit.DATE
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends = client.fetch_keyword_device_traffic(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")

API 호출 성공.


In [15]:
df_trends

,period,pc,mobile
0,2025-01-03,12.5,12.5
1,2025-01-07,25.0,25.0
2,2025-01-09,12.5,12.5
3,2025-01-15,12.5,50.0
4,2025-01-16,12.5,12.5
5,2025-01-21,25.0,12.5
6,2025-01-22,12.5,100.0
7,2025-01-28,25.0,12.5
8,2025-02-13,37.5,12.5
9,2025-02-19,50.0,12.5


## 2.3 키워드 내 성별 트렌드 조회

In [11]:
# ✅ category는 object list 형식이어야 함
keywords = "건강"

category_config = KeywordRequestConfig(
    start_date="2025-01-01",
    end_date="2025-03-15",
    category="50000006",
    keyword=keywords,
    time_unit=TimeUnit.MONTH,
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends = client.fetch_keyword_gender_traffic(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")

API 호출 성공.


In [13]:
df_trends

,period,female,male
0,2025-01-01,44.44444,38.88888
1,2025-02-01,33.33333,100.00000
2,2025-03-01,33.33333,27.77777


In [14]:
output_notebook()

p = figure(
    title="네이버 쇼핑 키워드 내 성별 트렌드 조회", 
    x_axis_type="datetime",
    height=600,
    width=1200,
    tools="pan,box_zoom,reset,save",
    sizing_mode="stretch_width"
)

palette = Category10[10]

# 연령대 레이블 매핑
gender_labels = {
    'female': 'female',
    'male': 'male'
}

# df의 period 컬럼을 x축으로 사용
for i, column in enumerate(sorted(df_trends.columns[1:])):
    source = ColumnDataSource(df_trends[['period', column]].rename(columns={column: 'ratio'}))
    gender_label = gender_labels.get(column, column)
    
    # ColumnDataSource에 legend_label 필드 추가
    source.data['legend_label'] = [gender_label] * len(source.data['period'])
    
    p.line(x='period', y='ratio', legend_label=gender_label,
           line_width=2, color=palette[i % 10], source=source)
    p.scatter(x='period', y='ratio', size=5, color=palette[i % 10], source=source)

hover = HoverTool(tooltips=[
    ('연령', '@legend_label'),
    ('비율', '@ratio{0.00}%'),
    ('일자', '@period{%F}')
], formatters={'@period': 'datetime'})

p.add_tools(hover)
p.xaxis.major_label_orientation = 0.8
p.legend.click_policy = "hide" 
p.legend.location = "top_left"
p.xaxis.axis_label = '날짜'
p.yaxis.axis_label = '검색 비율(%)'

show(p)

Loading BokehJS ...

#### 월별로 수집 (4/24)

In [ ]:
category_config = KeywordRequestConfig(
    start_date="2025-01-01",
    end_date="2025-03-15",
    category="50000006",
    keyword=keywords,
    time_unit=TimeUnit.MONTH,
)

In [17]:
# 월별 데이터 수집 및 병합
def fetch_monthly_data(keywords, start_date, end_date):
    merged_df = pd.DataFrame()
    date_ranges = generate_monthly_dates(start_date, end_date)

    for date_range in tqdm(date_ranges, total=len(date_ranges)):
        # print(f"수집 중: {date_range['start']} ~ {date_range['end']}")
        # API 요청 설정
        category_config = KeywordRequestConfig(
            start_date=date_range['start'],
            end_date=date_range['end'],
            category="50000006",
            keyword=keywords,
            time_unit=TimeUnit.MONTH
        )
    
        # API 요청
        temp = client.fetch_keyword_gender_traffic(category_config)
        merged_df = pd.concat([merged_df, temp])
    
    return merged_df

In [ ]:
["건강식", "다이어트", "샐러드", "간편식"]

In [18]:
df_health_food_gender = fetch_monthly_data("건강식", "2020-03-01", "2025-03-31")

  0%|          | 0/63 [00:00<?, ?it/s]

In [21]:
df_diet_gender = fetch_monthly_data("다이어트", "2020-03-01", "2025-03-31")
df_salad_gender = fetch_monthly_data("샐러드", "2020-03-01", "2025-03-31")
df_simple_food_gender = fetch_monthly_data("간편식", "2020-03-01", "2025-03-31")

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

In [25]:
df_health_food_gender.to_parquet("naverapi_result/df_health_food_gender.parquet", index=False)
df_diet_gender.to_parquet("naverapi_result/df_diet_gender.parquet", index=False)
df_salad_gender.to_parquet("naverapi_result/df_salad_gender.parquet", index=False)
df_simple_food_gender.to_parquet("naverapi_result/df_simple_food_gender.parquet", index=False)

In [20]:
# 성별 건강식 클릭 트렌드
plot_keyword_age_trends(df=df_health_food_gender, word="건강식")

Loading BokehJS ...

In [22]:
# 성별 다이어트 클릭 트렌드
plot_keyword_age_trends(df=df_diet_gender, word="다이어트")

Loading BokehJS ...

In [23]:
# 성별 샐러드 클릭 트렌드
plot_keyword_age_trends(df=df_salad_gender, word="샐러드")

Loading BokehJS ...

In [24]:
# 성별 간편식 클릭 트렌드
plot_keyword_age_trends(df=df_simple_food_gender, word="간편식")

Loading BokehJS ...

## 2.4 키워드 내 연령별 트렌드 조회

In [16]:
# ✅ category는 object list 형식이어야 함
keywords = "건강"

category_config = KeywordRequestConfig(
    start_date="2022-01-01",
    end_date="2025-04-01",
    category="50000006",
    keyword=keywords,
    time_unit=TimeUnit.MONTH,
    ages=["20", "30", "40"]
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends = client.fetch_keyword_age_traffic(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")

API 호출 성공.


In [17]:
df_trends

,period,10대,20대,30대,40대,50대,60대
0,2022-01-01,,2.10526,7.89473,5.26315,,
1,2022-02-01,,9.47368,4.21052,1.05263,,
2,2022-03-01,,1.57894,5.26315,1.57894,,
3,2022-04-01,,6.8421,8.42105,3.15789,,
4,2022-05-01,,4.21052,6.84210,4.21052,,
5,2022-06-01,,3.68421,8.94736,4.21052,,
6,2022-07-01,,1.05263,8.94736,0.52631,,
7,2022-08-01,,3.68421,2.63157,5.78947,,
8,2022-09-01,,8.94736,14.73684,3.68421,,
9,2022-10-01,,2.63157,11.57894,4.21052,,


In [18]:
output_notebook()

p = figure(
    title="네이버 쇼핑 카테고리별 검색 클릭 비율", 
    x_axis_type="datetime",
    height=600,
    width=1200,
    tools="pan,box_zoom,reset,save",
    sizing_mode="stretch_width"
)

palette = Category10[10]

# 연령대 레이블 매핑
age_labels = {
    '10': '10대',
    '20': '20대', 
    '30': '30대',
    '40': '40대',
    '50': '50대',
    '60': '60대'
}

# df의 period 컬럼을 x축으로 사용
for i, column in enumerate(sorted(df_trends.columns[1:])):
    source = ColumnDataSource(df_trends[['period', column]].rename(columns={column: 'ratio'}))
    age_label = age_labels.get(column, column)
    
    # ColumnDataSource에 legend_label 필드 추가
    source.data['legend_label'] = [age_label] * len(source.data['period'])
    
    p.line(x='period', y='ratio', legend_label=age_label,
           line_width=2, color=palette[i % 10], source=source)
    p.scatter(x='period', y='ratio', size=5, color=palette[i % 10], source=source)

hover = HoverTool(tooltips=[
    ('연령', '@legend_label'),
    ('비율', '@ratio{0.00}%'),
    ('일자', '@period{%F}')
], formatters={'@period': 'datetime'})

p.add_tools(hover)
p.xaxis.major_label_orientation = 0.8
p.legend.click_policy = "hide" 
p.legend.location = "top_left"
p.xaxis.axis_label = '날짜'
p.yaxis.axis_label = '검색 비율(%)'

show(p)

Loading BokehJS ...

In [15]:
# ✅ category는 object list 형식이어야 함
keywords = "건강"

category_config = KeywordRequestConfig(
    start_date="2023-01-01",
    end_date="2025-04-01",
    category="50000006",
    keyword=keywords,
    time_unit=TimeUnit.MONTH,
    ages=["20", "30", "40"]
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends_1 = client.fetch_keyword_age_traffic(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")
df_trends_1

API 호출 성공.


,period,10대,20대,30대,40대,50대,60대
0,2023-01-01,,2.10526,5.26315,3.15789,,
1,2023-02-01,,5.26315,10.00000,5.26315,,
2,2023-03-01,,3.15789,14.73684,6.8421,,
3,2023-04-01,,5.26315,15.78947,1.05263,,
4,2023-05-01,,23.15789,16.31578,1.05263,,
5,2023-06-01,,5.26315,8.42105,3.68421,,
6,2023-07-01,,11.57894,2.63157,98.42105,,
7,2023-08-01,,3.15789,6.84210,3.15789,,
8,2023-09-01,,2.63157,5.78947,1.05263,,
9,2023-10-01,,1.57894,0.52631,40.52631,,


In [42]:
# ✅ category는 object list 형식이어야 함
keywords = "건강식"

category_config = KeywordRequestConfig(
    start_date="2022-04-01",
    end_date="2025-04-01",
    category="50000006",
    keyword=keywords,
    time_unit=TimeUnit.MONTH,
    ages=["20", "30", "40"]
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends_1 = client.fetch_keyword_age_traffic(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")
df_trends_1

API 호출 성공.


,period,10대,20대,30대,40대,50대,60대
0,2022-04-01,,41.95804,57.34265,47.55244,,
1,2022-05-01,,41.95804,61.53846,60.13986,,
2,2022-06-01,,49.65034,100.00000,60.83916,,
3,2022-07-01,,58.04195,76.22377,70.62937,,
4,2022-08-01,,58.74125,87.41258,73.42657,,
5,2022-09-01,,60.83916,61.53846,39.16083,,
6,2022-10-01,,45.45454,68.53146,72.72727,,
7,2022-11-01,,34.96503,51.74825,42.65734,,
8,2022-12-01,,32.16783,32.86713,29.37062,,
9,2023-01-01,,39.86013,62.93706,52.44755,,


In [43]:
df_trends_1.drop(['10대'], axis=1, inplace=True)

In [44]:
output_notebook()

p = figure(
    title="네이버 쇼핑 카테고리별 검색 클릭 비율 (건강식)", 
    x_axis_type="datetime",
    height=600,
    width=1200,
    tools="pan,box_zoom,reset,save",
    sizing_mode="stretch_width"
)

palette = Category10[10]

# 연령대 레이블 매핑
age_labels = {
    '10': '10대',
    '20': '20대', 
    '30': '30대',
    '40': '40대',
    '50': '50대',
    '60': '60대'
}

# df의 period 컬럼을 x축으로 사용
for i, column in enumerate(sorted(df_trends_1.columns[1:])):
    source = ColumnDataSource(df_trends_1[['period', column]].rename(columns={column: 'ratio'}))
    age_label = age_labels.get(column, column)
    
    # ColumnDataSource에 legend_label 필드 추가
    source.data['legend_label'] = [age_label] * len(source.data['period'])
    
    p.line(x='period', y='ratio', legend_label=age_label,
           line_width=2, color=palette[i % 10], source=source)
    p.scatter(x='period', y='ratio', size=5, color=palette[i % 10], source=source)

hover = HoverTool(tooltips=[
    ('연령', '@legend_label'),
    ('비율', '@ratio{0.00}%'),
    ('일자', '@period{%F}')
], formatters={'@period': 'datetime'})

p.add_tools(hover)
p.xaxis.major_label_orientation = 0.8
p.legend.click_policy = "hide" 
p.legend.location = "top_left"
p.xaxis.axis_label = '날짜'
p.yaxis.axis_label = '검색 비율(%)'

show(p)

Loading BokehJS ...

In [88]:
# ✅ category는 object list 형식이어야 함
keywords = "간편식"

category_config = KeywordRequestConfig(
    start_date="2022-01-01",
    end_date="2025-04-01",
    category="50000006",
    keyword=keywords,
    time_unit=TimeUnit.WEEK,
    ages=["20", "30", "40", "50", "60"]
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends_2 = client.fetch_keyword_age_traffic(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")
# non_numeric_cols = df_trends_2.select_dtypes(include=['object']).columns
# df_trends_2.drop(non_numeric_cols, axis=1, inplace=True)

API 호출 성공.


In [8]:
def plot_keyword_age_trends(df, word):
    output_notebook()

    p = figure(
        title=f"네이버 쇼핑 카테고리별 검색 클릭 비율 ({word})", 
        x_axis_type="datetime",
        height=600,
        width=1200,
        tools="pan,box_zoom,reset,save",
        sizing_mode="stretch_width"
    )

    palette = Category10[10]

    # 연령대 레이블 매핑 - 실제 데이터프레임에 있는 컬럼만 포함
    age_labels = {}
    for col in df.columns[1:]:  # period 컬럼 제외
        if col in ['20', '30', '40']:  # 실제 존재하는 연령대 컬럼만 매핑
            age_labels[col] = f'{col}대'

    # df의 period 컬럼을 x축으로 사용
    for i, column in enumerate(sorted(df.columns[1:])):
        source = ColumnDataSource(df[['period', column]].rename(columns={column: 'ratio'}))
        age_label = age_labels.get(column, column)
        
        # ColumnDataSource에 legend_label 필드 추가
        source.data['legend_label'] = [age_label] * len(source.data['period'])
        
        p.line(x='period', y='ratio', legend_label=age_label,
            line_width=2, color=palette[i % 10], source=source)
        p.scatter(x='period', y='ratio', size=5, color=palette[i % 10], source=source)

    hover = HoverTool(tooltips=[
        ('연령', '@legend_label'),
        ('비율', '@ratio{0.00}%'),
        ('일자', '@period{%F}')
    ], formatters={'@period': 'datetime'})

    p.add_tools(hover)
    p.xaxis.major_label_orientation = 0.8
    p.legend.click_policy = "hide" 
    p.legend.location = "top_left"
    p.xaxis.axis_label = '날짜'
    p.yaxis.axis_label = '검색 비율(%)'

    show(p)

In [89]:
df_trends_2

,period,10대,20대,30대,40대,50대,60대
0,2021-12-27,,60.43478,67.82608,54.34782,22.17391,3.04347
1,2022-01-03,,77.39130,100.00000,77.39130,26.52173,4.78260
2,2022-01-10,,59.56521,95.21739,84.78260,31.73913,7.82608
3,2022-01-17,,56.08695,93.04347,67.39130,25.21739,8.26086
4,2022-01-24,,46.08695,50.00000,50.43478,11.30434,5.21739
...,...,...,...,...,...,...,...
166,2025-03-03,,27.82608,42.17391,27.82608,20.86956,5.21739
167,2025-03-10,,15.21739,33.47826,46.52173,23.04347,2.17391
168,2025-03-17,,20.00000,25.21739,33.47826,18.69565,11.73913
169,2025-03-24,,19.13043,27.82608,31.30434,18.69565,3.47826


In [41]:
plot_keyword_age_trends(df_trends_2, word="간편식")

Loading BokehJS ...

- 2023년도 1분기에는 40대의 "건강식" 키워드 검색이 많았으나, 점점 가면 갈수록 2~30대의 키워드 검색이 많아지는 것을 볼 수 있음
- 건강식으로 제품을 포지셔닝한다면 2~30대를 대상으로 진행하는 데 설득의 근거가 될 수 있을 듯.
- 월별로 데이터를 받아서 보는 것 보다 주별로 데이터를 받아서 보는 것이 이러한 면이 더 부각됨.

In [41]:
# ✅ category는 object list 형식이어야 함
keywords = "간편식"

category_config = KeywordRequestConfig(
    start_date="2020-01-01",
    end_date="2025-04-01",
    category="50000006",
    keyword=keywords,
    time_unit=TimeUnit.WEEK,
    ages=["20", "30", "40"]
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends_week_5y = client.fetch_keyword_age_traffic(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")
df_trends_week_5y.drop(['10대', '50대', '60대'], axis=1, inplace=True)
df_trends_week_5y

API 호출 성공.


,period,10대,20대,30대,40대,50대,60대
0,2019-12-30,,26.11940,43.28358,36.56716,,
1,2020-01-06,,39.55223,51.99004,44.77611,,
2,2020-01-13,,34.07960,38.05970,32.33830,,
3,2020-01-20,,16.91542,29.35323,22.13930,,
4,2020-01-27,,32.83582,52.48756,38.55721,,
...,...,...,...,...,...,...,...
270,2025-03-03,,15.92039,24.12935,15.92039,,
271,2025-03-10,,8.70646,19.15422,26.61691,,
272,2025-03-17,,11.44278,14.42786,19.15422,,
273,2025-03-24,,10.94527,15.92039,17.91044,,


In [42]:
df_trends_week_5y.drop(['10대', '50대', '60대'], axis=1, inplace=True)

In [43]:
output_notebook()

p = figure(
    title="네이버 쇼핑 카테고리별 검색 클릭 비율", 
    x_axis_type="datetime",
    height=600,
    width=1200,
    tools="pan,box_zoom,reset,save",
    sizing_mode="stretch_width"
)

palette = Category10[10]

# 연령대 레이블 매핑
age_labels = {
    '10': '10대',
    '20': '20대', 
    '30': '30대',
    '40': '40대',
    '50': '50대',
    '60': '60대'
}

# df의 period 컬럼을 x축으로 사용
for i, column in enumerate(sorted(df_trends_week_5y.columns[1:])):
    source = ColumnDataSource(df_trends_week_5y[['period', column]].rename(columns={column: 'ratio'}))
    age_label = age_labels.get(column, column)
    
    # ColumnDataSource에 legend_label 필드 추가
    source.data['legend_label'] = [age_label] * len(source.data['period'])
    
    p.line(x='period', y='ratio', legend_label=age_label,
           line_width=2, color=palette[i % 10], source=source)
    p.scatter(x='period', y='ratio', size=5, color=palette[i % 10], source=source)

hover = HoverTool(tooltips=[
    ('연령', '@legend_label'),
    ('비율', '@ratio{0.00}%'),
    ('일자', '@period{%F}')
], formatters={'@period': 'datetime'})

p.add_tools(hover)
p.xaxis.major_label_orientation = 0.8
p.legend.click_policy = "hide" 
p.legend.location = "top_left"
p.xaxis.axis_label = '날짜'
p.yaxis.axis_label = '검색 비율(%)'

show(p)

Loading BokehJS ...

In [79]:
# ✅ category는 object list 형식이어야 함
keywords = "건강식"

category_config = KeywordRequestConfig(
    start_date="2025-03-01",
    end_date="2025-03-31",
    category="50000006",
    keyword=keywords,
    time_unit=TimeUnit.DATE,
    ages=["20", "30", "40", "50", "60"]
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends_1 = client.fetch_keyword_age_traffic(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")
# non_numeric_cols = df_trends_1.select_dtypes(include=['object']).columns
# df_trends_1.drop(non_numeric_cols, axis=1, inplace=True)
df_trends_1

API 호출 성공.


,period,10대,20대,30대,40대,50대,60대
0,2025-03-01,,14.28571,14.28571,28.57142,,
1,2025-03-02,,14.28571,28.57142,,28.57142,
2,2025-03-03,,14.28571,57.14285,71.42857,,
3,2025-03-04,,28.57142,28.57142,42.85714,,
4,2025-03-05,,,57.14285,42.85714,,
5,2025-03-06,,,14.28571,14.28571,14.28571,
6,2025-03-07,,14.28571,,42.85714,,
7,2025-03-08,,14.28571,28.57142,,,
8,2025-03-09,,14.28571,14.28571,,,
9,2025-03-10,,,14.28571,,,


In [80]:
df_trends_1.dtypes

period    datetime64[ns]
10대               object
20대               object
30대               object
40대               object
50대               object
60대               object
dtype: object

In [ ]:
2023.01.01 - 2023.12.31 // 2024.01.01 - 2024.12.31

In [71]:
df_trends_1

,period
0,2025-01-01
1,2025-01-02
2,2025-01-03
3,2025-01-04
4,2025-01-05
...,...
84,2025-03-27
85,2025-03-28
86,2025-03-29
87,2025-03-30


In [81]:
plot_keyword_age_trends(df_trends_1, word="건강식")

Loading BokehJS ...

In [104]:
keywords = ["건강식", "샐러드"]

category_config = KeywordRequestConfig(
    start_date="2025-01-01",
    end_date="2025-03-31",
    category="50000006",
    keyword=keywords,
    time_unit=TimeUnit.DATE,
    ages=["20", "30", "40", "50", "60"]
)

# 6. 에러 처리를 포함한 API 호출
try:
    df_trends_1 = client.fetch_keyword_age_traffic(category_config)
    print("API 호출 성공.")
except Exception as e:
    print(f"API 호출 중 오류 발생: {str(e)}")
# non_numeric_cols = df_trends_1.select_dtypes(include=['object']).columns
# df_trends_1.drop(non_numeric_cols, axis=1, inplace=True)
df_trends_2

API 호출 중 오류 발생: API 요청 실패: 400 Client Error: Bad Request for url: https://openapi.naver.com/v1/datalab/shopping/category/keyword/age


,period,10대,20대,30대,40대,50대,60대
0,2021-12-27,,60.43478,67.82608,54.34782,22.17391,3.04347
1,2022-01-03,,77.39130,100.00000,77.39130,26.52173,4.78260
2,2022-01-10,,59.56521,95.21739,84.78260,31.73913,7.82608
3,2022-01-17,,56.08695,93.04347,67.39130,25.21739,8.26086
4,2022-01-24,,46.08695,50.00000,50.43478,11.30434,5.21739
...,...,...,...,...,...,...,...
166,2025-03-03,,27.82608,42.17391,27.82608,20.86956,5.21739
167,2025-03-10,,15.21739,33.47826,46.52173,23.04347,2.17391
168,2025-03-17,,20.00000,25.21739,33.47826,18.69565,11.73913
169,2025-03-24,,19.13043,27.82608,31.30434,18.69565,3.47826


### 월별로 수집 (4/23)

In [4]:
# 월별 데이터 수집 및 병합
def fetch_monthly_data(keywords, start_date, end_date):
    merged_df = pd.DataFrame()
    date_ranges = generate_monthly_dates(start_date, end_date)

    for date_range in tqdm(date_ranges, total=len(date_ranges)):
        # print(f"수집 중: {date_range['start']} ~ {date_range['end']}")
        # API 요청 설정
        category_config = KeywordRequestConfig(
            start_date=date_range['start'],
            end_date=date_range['end'],
            category="50000006",
            keyword=keywords,
            time_unit=TimeUnit.MONTH,
            ages=["20", "30", "40", "50", "60"]
        )
    
        # API 요청
        temp = client.fetch_keyword_age_traffic(category_config)
        merged_df = pd.concat([merged_df, temp])
    
    return merged_df

In [5]:
df_health_food = fetch_monthly_data("건강식", "2020-03-01", "2025-03-31")

  0%|          | 0/61 [00:00<?, ?it/s]

In [6]:
df_health_food.to_parquet("naverapi_result/건강식_5y_month.parquet", index=False)

In [7]:
df_diet = fetch_monthly_data("다이어트", "2020-03-01", "2025-03-31")

  0%|          | 0/61 [00:00<?, ?it/s]

In [8]:
df_diet.to_parquet("naverapi_result/다이어트_5y_month.parquet", index=False)

In [9]:
df_salad = fetch_monthly_data("샐러드", "2020-03-01", "2025-03-31")
df_salad.to_parquet("naverapi_result/샐러드_5y_month.parquet", index=False)

  0%|          | 0/61 [00:00<?, ?it/s]

In [10]:
df_simple_food = fetch_monthly_data("간편식", "2020-03-01", "2025-03-31")
df_simple_food.to_parquet("naverapi_result/간편식_5y_month.parquet", index=False)

  0%|          | 0/61 [00:00<?, ?it/s]

In [31]:
df_pumkin = fetch_monthly_data("단호박", "2020-03-01", "2025-03-31")
df_pumkin.to_parquet("naverapi_result/단호박_5y_month.parquet", index=False)

  0%|          | 0/61 [00:00<?, ?it/s]

OSError: Cannot save file into a non-existent directory: 'naverapi_rsult'

In [13]:
plot_keyword_age_trends(df=df_health_food, word="건강식")

Loading BokehJS ...

In [14]:
plot_keyword_age_trends(df=df_diet, word="다이어트")

Loading BokehJS ...

In [15]:
plot_keyword_age_trends(df=df_salad, word="샐러드")

Loading BokehJS ...

In [16]:
plot_keyword_age_trends(df=df_simple_food, word="간편식")

Loading BokehJS ...

In [32]:
plot_keyword_age_trends(df=df_pumkin, word="단호박")

Loading BokehJS ...

In [34]:
df_sweet_potato = fetch_monthly_data("고구마", "2020-03-01", "2025-03-31")
df_sweet_potato.to_parquet("naverapi_result/고구마_5y_month.parquet", index=False)
plot_keyword_age_trends(df=df_sweet_potato, word="고구마")

  0%|          | 0/61 [00:00<?, ?it/s]

Loading BokehJS ...

In [5]:
df_asai_bowl = fetch_monthly_data("아사이볼", "2020-03-01", "2025-03-31")
df_asai_bowl.to_parquet("naverapi_result/아사이볼_5y_month.parquet", index=False)
plot_keyword_age_trends(df=df_asai_bowl, word="아사이볼")

  0%|          | 0/61 [00:00<?, ?it/s]

Loading BokehJS ...

In [6]:
df_broccoli = fetch_monthly_data("브로콜리", "2020-03-01", "2025-03-31")
df_broccoli.to_parquet("naverapi_result/브로콜리_5y_month.parquet", index=False)
plot_keyword_age_trends(df=df_broccoli, word="브로콜리")

  0%|          | 0/61 [00:00<?, ?it/s]

Loading BokehJS ...

In [7]:
df_chicken_breast = fetch_monthly_data("닭가슴살", "2020-03-01", "2025-03-31")
df_chicken_breast.to_parquet("naverapi_result/닭가슴살_5y_month.parquet", index=False)
plot_keyword_age_trends(df=df_chicken_breast, word="닭가슴살")

  0%|          | 0/61 [00:00<?, ?it/s]

Loading BokehJS ...

In [8]:
df_asai_bowl

,period,10대,20대,30대,40대,50대,60대
0,2020-03-01,0,60.86956,100.0,4.34782,0.00000,0.00000
0,2020-04-01,0,44.44444,100.0,0.00000,0.00000,0.00000
0,2020-05-01,0,57.14285,100.0,0.00000,0.00000,0.00000
0,2020-06-01,0,50.00000,100.0,0.00000,0.00000,0.00000
0,2020-07-01,0,100.00000,87.5,12.50000,0.00000,0.00000
...,...,...,...,...,...,...,...
0,2024-11-01,0,31.56342,100.0,60.47197,12.97935,1.17994
0,2024-12-01,0,37.60000,100.0,30.40000,22.00000,4.40000
0,2025-01-01,0,44.81865,100.0,35.49222,4.40414,0.77720
0,2025-02-01,0,34.65909,100.0,35.22727,9.65909,3.69318


In [38]:
df_asai_bowl.dtypes

period    datetime64[ns]
10대               object
20대              float64
30대              float64
40대               object
50대               object
60대               object
dtype: object

['갈치', '피마늘', '포도', '양파', '새송이버섯', '브로콜리', '배', '딸기', '사과', '단감', '고구마', '굴', '레몬', '망고', '감자', '느타리버섯', '당근', '감귤', '토마토', '체리']

---

🥗 1. 브로콜리 고구마 닭가슴살 볼
	•	브로콜리, 고구마, 양파, 레몬 드레싱
	•	👉 저탄수 + 고단백 식단, 렌지 3분 조리 가능
	•	🧠 타겟: 다이어트/헬스족, 직장인 점심

⸻

🍚 2. 피마늘 감자볶음밥
	•	감자, 양파, 피마늘, 당근 + 곁들임 계란
	•	👉 장기 보존 가능한 냉동 볶음밥 가능
	•	🧠 타겟: 혼밥족, 기숙사/자취생

⸻

🍱 3. 굴두부 된장보울
	•	굴, 양파, 느타리버섯, 단감 슬라이스 곁들임
	•	👉 전자렌지 보울형 국물 도시락
	•	🧠 타겟: 한식 선호 3040 여성, 시니어

⸻

🥣 4. 레몬 닭가슴살 샐러드
	•	브로콜리, 레몬, 사과, 토마토, 새송이버섯, 오일 드레싱
	•	👉 저지방 저열량, 비건 옵션 가능
	•	🧠 타겟: 운동 전후 간식, 건강식 관심층

⸻

🍲 5. 양파 갈치조림 밀키트
	•	갈치, 양파, 당근, 피마늘, 고구마 (조림용)
	•	👉 냉장 5일/냉동 보존 가능 밀키트 구성
	•	🧠 타겟: 중장년 가족형 식사 구성

⸻

🍧 6. 망고-체리-딸기 냉동 스무디팩
	•	딸기, 체리, 망고, 레몬 조각 → 믹서용 키트
	•	👉 비가열, 냉동 유통 / 1인용 건강 디저트
	•	🧠 타겟: 2030 여성층, 홈카페족

⸻

🍞 7. 단감 & 사과 요거트볼
	•	단감, 사과, 레몬즙, 요거트, 견과
	•	👉 아침식, 간식, 무설탕 디저트
	•	🧠 타겟: 건강 간식, 키즈 간편식

1. 온라인 -> 30대 / 건강식
2. NaverAPI (건강식, 샐러드, 간편식, 다이어트) -> 40대, 30대
3. 1주차 (변동성이 적은 품목 20개)

---

4. 뭘 만들 수 있을까 -> 10000개의 레시피에서 위에 들어간 리스트의 품목이 들어가는 요리(키워드)로 삼아서 트렌드 분석(10000개의 레시피 조회수 + 네이버 API)을 해본다. (LLM 대체 가능)
5. LLM (ChatGPT, Gemini) -> 
6. 리스트 도출-> 
7. 40대, 30대가 선호하는 품목이 들어간 메뉴로 선정을 해 보겠다. -> 
8. 시각화 자료들이 근거로 사용

##### previous

In [93]:
# 일별 2024-03-01 ~ 2025-03-31 (12번)
plot_keyword_age_trends(merged_df, word="건강식")

Loading BokehJS ...

In [18]:
# 일별 2024-03-01 ~ 2025-03-31 (12번)
plot_keyword_age_trends(df_1y, word="건강식")

Loading BokehJS ...

In [95]:
plot_keyword_age_trends(merged_df_week, word="건강식")

Loading BokehJS ...

In [97]:
plot_keyword_age_trends(merged_df_month, word="건강식")

Loading BokehJS ...

In [22]:
plot_keyword_age_trends(t, word="간편식")

Loading BokehJS ...

In [100]:
# 간편식
plot_keyword_age_trends(merged_df_month, word="간편식")

Loading BokehJS ...

In [40]:
# 건강식
plot_keyword_age_trends(merged_df_month, word="건강식_5년")

Loading BokehJS ...

In [83]:
plot_keyword_age_trends(df_trends_2, word="건강식")

Loading BokehJS ...

In [47]:
plot_keyword_age_trends(df_trends_1, word="샐러드")

Loading BokehJS ...

# ETC

In [ ]:
[
    {"name": "축산물", "param": ["50000145"]},
    {"name": "수산물", "param": ["50000159"]},
    {"name": "농산물", "param": ["50000160"]},
    {"name": "반찬", "param": ["50000146"]},
    {"name": "김치", "param": ["50000147"]},
    {"name": "음료", "param": ["50000148"]},
    {"name": "과자/베이커리", "param": ["50000149"]},
    {"name": "유가공품", "param": ["50000150"]},
    {"name": "냉동/간편조리식품", "param": ["50000026"]},
    {"name": "건강식품", "param": ["50000023"]},
    {"name": "다이어트식품", "param": ["50000024"]},
    {"name": "통조림/캔", "param": ["50011940"]},
    {"name": "제과/제빵재료", "param": ["5001246"]},
    {"name": "조미료", "param": ["50012520"]},
    {"name": "식용유/오일", "param": ["50012620"]},
    {"name": "소스/드레싱", "param": ["50012782"]},
    {"name": "가루/분말류", "param": ["50013360"]},
    {"name": "잼/시럽", "param": ["50013520"]},
    {"name": "라면/면류", "param": ["50013940"]},
    {"name": "장류", "param": ["50013881"]},
    {"name": "밀키트", "param": ["50014240"]},
    {"name": "주류", "param": ["50014520"]}
]

In [38]:
output_notebook()

p = figure(
    title="네이버 쇼핑 카테고리별 검색 비율",
    x_axis_type="datetime", 
    height=600,
    width=1200,
    tools="pan,box_zoom,reset,save"
)

palette = Category10[10]

for i, item in enumerate(sorted(df['group'].unique())):
    subset = df[df['group'] == item].copy().sort_values('date')
    source = ColumnDataSource(subset)

    p.line(x='date', y='ratio', legend_label=item,
           line_width=2, color=palette[i % 10], source=source)
    p.scatter(x='date', y='ratio', size=5, color=palette[i % 10], source=source)

hover = HoverTool(tooltips=[
    ('품목', '@group'),
    ('비율', '@ratio{0.00}%'),  # 퍼센트 포맷
    ('일자', '@date{%F}')
], formatters={'@date': 'datetime'})

p.add_tools(hover)
p.xaxis.major_label_orientation = 0.8
p.legend.click_policy = "hide"
p.legend.location = "top_left"
p.xaxis.axis_label = '날짜'
p.yaxis.axis_label = '검색 비율(%)'

show(p)


Loading BokehJS ...